### The evolutionary algorithm is an adaptation of [MourvanZhou's evolutionary algorithm code](https://github.com/MorvanZhou/Evolutionary-Algorithm/blob/master/tutorial-contents/Genetic%20Algorithm/Genetic%20Algorithm%20Basic.py) ###

### Squashed entanglement of a state ###

In [8]:
from qutip import *
from scipy import arcsin, sqrt, pi
import numpy as np
import scipy.sparse as sp
from qutip.qobj import Qobj
%matplotlib
import matplotlib.pyplot as plt
import itertools
import copy

Using matplotlib backend: Qt5Agg


Defining some fuctions

In [2]:
#rhoABE is a tripartite density matrix with the third subsystem being the ancilla (ancilla is bi-partite)
def cond_entropy_mutual(rhoABE):  
    AE = rhoABE.ptrace([0, 2, 3]);
    BE = rhoABE.ptrace([1, 2, 3]);
    ABE = rhoABE;
    E = rhoABE.ptrace([2, 3]);
    return entropy_vn(AE,2) + entropy_vn(BE,2) - entropy_vn(ABE,2) - entropy_vn(E,2); 


def rand_herm_mod(N, X, Y, density=1, dims=None):      #this is a modified version of the rand_herm function from QuTip
    if dims:
        _check_dims(dims, N, N)
    # to get appropriate density of output
    # Hermitian operator must convert via:
    herm_density = 2.0 * arcsin(density) / pi

    X_int = sp.rand(N, N, herm_density, format='csr')
    X_int.data = X - 0.5
    Y_int = X_int.copy()
    Y_int.data = 1.0j * Y - (0.5 + 0.5j)
    X_int = X_int + Y_int
    X_int.sort_indices()
    X_int = Qobj(X_int)
    if dims:
        return Qobj((X_int + X_int.dag()) / 2.0, dims=dims, shape=[N, N])
    else:
        return Qobj((X_int + X_int.dag()) / 2.0)
    

def rand_unitary_mod(N, X, Y, density=1, dims=None):    #this is a modified version of the rand_unitary function from QuTip
    #if dims:
    #    _check_dims(dims, N, N)
    U = (-1.0j * rand_herm_mod(N, X, Y, density)).expm()
    U.data.sort_indices()
    if dims:
        return Qobj(U, dims=dims, shape=[N, N])
    else:
        return Qobj(U)


    
def _check_dims(dims, N1, N2):   #this function is taken directly from QuTip
    if len(dims) != 2:
        raise Exception("Qobj dimensions must be list of length 2.")
    if (not isinstance(dims[0], list)) or (not isinstance(dims[1], list)):
        raise TypeError(
            "Qobj dimension components must be lists. i.e. dims=[[N],[N]]")
    if np.prod(dims[0]) != N1 or np.prod(dims[1]) != N2:
        raise ValueError("Qobj dimensions must match matrix shape.")
    if len(dims[0]) != len(dims[1]):
        raise TypeError("Qobj dimension components must have same length.")

The adapted genetic algorithm. 
Each individual in the population is a list of two lists X and Y that seed a unitary.

In [3]:
#Adding the ancillary system
F_dim = 2
#E_dim = 2  #Dimension of the ancilla system
A_dim = 3
B_dim = 3
#Writing down the bi-partite system's density matrix. Different states.

#A random state. Changes every time this cell is executed.
#rhoAB = rand_dm(4, 0.5, dims=[[2,2], [2,2]]);

#The state in (page 4, Christandl and Winter 2003)
ket1 = (tensor(basis(3,1),basis(3,2)) - tensor(basis(3,2),basis(3,1))).unit()
ket2 = (tensor(basis(3,2),basis(3,0)) - tensor(basis(3,0),basis(3,2))).unit()
ket3 = (tensor(basis(3,0),basis(3,1)) - tensor(basis(3,1),basis(3,0))).unit()
rhoAB = (ket2dm(ket1) + ket2dm(ket2) + ket2dm(ket3)).unit()


ket_pure_ABE = (tensor(ket1, ket1) + tensor(ket2, ket2) + tensor(ket3, ket3)).unit()
#print(ket_pure_ABE)

temp_psi_ABEF = tensor(ket_pure_ABE, basis(F_dim,0))
temp_rho_ABEF = ket2dm(temp_psi_ABEF)

#print(temp_rho_ABEF)

In [9]:
N = A_dim * B_dim * F_dim
#rhoE = ket2dm(basis(E_dim,0)) #|0...0><0...0|
#rhoABE = tensor(rhoAB,rhoE)

DNA_SIZE = 30           # DNA length   # size of each number in the lists X and Y
POP_SIZE = 100           # population size
CROSS_RATE = 0.001         # mating probability (DNA crossover)
MUTATION_RATE = 0.001    # mutation probability
N_GENERATIONS = 1000
#X_BOUND = [0, 1]         # x upper and lower bounds. Modified: changed the bound


def F(Us): #modified: out F is the conditional mutual information. FIXED
    return 0.5 * np.array([cond_entropy_mutual((tensor(identity(A_dim), identity(B_dim), U) * temp_rho_ABEF * tensor(identity(A_dim), identity(B_dim), U).dag()).ptrace([0,1,2,3]) ) for U in Us]) 
    #return 0.5 * np.array([cond_entropy_mutual(( tensor(identity(A_dim) ,identity(B_dim), identity(A_dim), identity(B_dim), identity(F_dim)) * temp_rho_ABEF * 
    #                                    tensor(identity(A_dim) ,identity(B_dim), identity(A_dim), identity(B_dim), identity(F_dim))).ptrace([0,1,2,3]) ) for U in Us]) 

    
# find non-zero fitness for selection. FIXED
def get_fitness(pred): return 1/pred - 1/np.max(pred)  #modified: fitness is highest when cmi is lowest

# convert binary DNA to decimal and normalize it to a range(0, 1). Modified: takes unitary seeding list. returns a Unitary
def translateDNA(pop): # FIXED
    Z = pop.dot(2 ** np.arange(DNA_SIZE)[::-1]) / (2**DNA_SIZE-1)
    return np.array([rand_unitary_mod(N, Z[i,0], Z[i,1], density=1, dims=[[A_dim,B_dim, F_dim], [A_dim,B_dim, F_dim]])  
                     for i in range(POP_SIZE)])

# nature selection wrt pop's fitness. Not modified. Should work as it is. FIXED
def select(pop, fitness):    
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True, p=fitness/fitness.sum())
    return pop[idx]

# mating process (genes crossover).
def crossover_and_mutate(individual, pop, mutate=False):
    #crossover
    if np.random.rand() < CROSS_RATE:
        i_ = np.random.randint(0, POP_SIZE, size=1)[0]                        # select another individual from pop
        cross_points = np.random.randint(0, 2, size=DNA_SIZE*2*N**2).astype(bool).reshape((2,N**2,DNA_SIZE))# choose crossover points
        individual[cross_points] = pop[i_][cross_points]
        
    #mutate
    if mutate:
        x = np.random.choice([0, 1], size=DNA_SIZE*2*N**2, 
                             p=[1-MUTATION_RATE, MUTATION_RATE]).astype(bool).reshape((2,N**2,DNA_SIZE))
        individual[x] = np.abs(individual[x] - 1)  #flip the bits
        
    return individual


pop = np.random.randint(2, size=([POP_SIZE, 2, N**2, DNA_SIZE]))

#cmi_list = []

plt.ion()
fig, ax = plt.subplots()
gen, cmi_list = [],[]
ax.scatter(gen,cmi_list)
plt.xlim(0,N_GENERATIONS)
plt.ylim(0.78,0.8)
plt.plot([i for i in range(N_GENERATIONS)], [0.5*np.log2(3) for j in range(N_GENERATIONS)])
plt.draw()
plt.xlabel("Generations")
plt.ylabel("Squashed entanglement estimate")

for _ in range(N_GENERATIONS):
    F_values = F(translateDNA(pop))    # compute function value by extracting DNA
    #F_values = [F(translateDNA(np.array(ind))) for ind in pop]
    cmi_list.append(np.max(F_values))
    ax.scatter(_, cmi_list[-1], c='red')
    plt.pause(0.05)


    # GA part (evolution)
    fitness = get_fitness(F_values) #FIXED
    pop = select(pop, fitness)
    pop_copy = pop.copy()
    for parent in pop:
        child = crossover_and_mutate(parent, pop_copy, len(cmi_list)>1 and cmi_list[-1]==cmi_list[-2]) 
        parent[:] = child       # parent is replaced by its child

plt.ioff()

C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
C:\Users\basse\Anaconda3\envs\qutip\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in less


KeyboardInterrupt: 